In [1]:
# import kagglehub
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import re
import string
import asyncio
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
# from googletrans import Translator, constants
# from pprint import pprint
from translate import Translator
from ydata_profiling import ProfileReport

# Download latest version
# path = kagglehub.dataset_download("canggih/jog-description-and-salary-in-indonesia")

# print("Path to dataset files:", path)

In [2]:
df = pd.read_csv('../../data/all.csv', delimiter='|')
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN


In [3]:
df = df.drop(columns=['id', 'location', 'salary_currency', 'employment_type', 'job_function', 'job_benefits', 'company_process_time',
                     'company_size', 'company_industry', 'salary'])
df.head()

,job_title,career_level,experience_level,education_level,job_description
0,Facility Maintenance & Smart Warehouse Manager,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Deskripsi PekerjaanRequirements :D3/SI from re...
1,Procurement Department Head,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Job Role: 1. Responsible for material availabi...
2,SALES ADMIN,Supervisor/Koordinator,4 tahun,Sarjana (S1),Internal Sales & AdminJob Description :We are ...
3,City Operation Lead Shopee Express (Cirebon),Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Job Description:Responsible for HSE implementa...
4,Japanese Interpreter,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Overview: Our clients is manufacture for autom...


In [4]:
for col in df:
    df[col] = df[col].str.lower()

df.head()

,job_title,career_level,experience_level,education_level,job_description
0,facility maintenance & smart warehouse manager,manajer/asisten manajer,5 tahun,"sertifikat professional, d3 (diploma), d4 (dip...",deskripsi pekerjaanrequirements :d3/si from re...
1,procurement department head,manajer/asisten manajer,5 tahun,"sarjana (s1), diploma pascasarjana, gelar prof...",job role: 1. responsible for material availabi...
2,sales admin,supervisor/koordinator,4 tahun,sarjana (s1),internal sales & adminjob description :we are ...
3,city operation lead shopee express (cirebon),supervisor/koordinator,5 tahun,"sarjana (s1), diploma pascasarjana, gelar prof...",job description:responsible for hse implementa...
4,japanese interpreter,pegawai (non-manajemen & non-supervisor),2 tahun,"sertifikat professional, d3 (diploma), d4 (dip...",overview: our clients is manufacture for autom...


In [5]:
df.isnull().sum()

job_title              0
career_level           0
experience_level    4541
education_level        0
job_description        1
dtype: int64

In [6]:
df.dropna(inplace=True)
df.isnull().sum()

job_title           0
career_level        0
experience_level    0
education_level     0
job_description     0
dtype: int64

In [7]:
for col in df:
    if col == 'experience_level':
        continue
    df[col] = df[col].apply(lambda x: re.sub(r'\w*\d\w*',' ', x))

df.head()

,job_title,career_level,experience_level,education_level,job_description
0,facility maintenance & smart warehouse manager,manajer/asisten manajer,5 tahun,"sertifikat professional, (diploma), (diplo...",deskripsi pekerjaanrequirements : /si from rep...
1,procurement department head,manajer/asisten manajer,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job role: . responsible for material availabi...
2,sales admin,supervisor/koordinator,4 tahun,sarjana ( ),internal sales & adminjob description :we are ...
3,city operation lead shopee express (cirebon),supervisor/koordinator,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job description:responsible for hse implementa...
4,japanese interpreter,pegawai (non-manajemen & non-supervisor),2 tahun,"sertifikat professional, (diploma), (diplo...",overview: our clients is manufacture for autom...


In [8]:
df['job_description']=df['job_description'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))
df.head()

,job_title,career_level,experience_level,education_level,job_description
0,facility maintenance & smart warehouse manager,manajer/asisten manajer,5 tahun,"sertifikat professional, (diploma), (diplo...",deskripsi pekerjaanrequirements si from rep...
1,procurement department head,manajer/asisten manajer,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job role responsible for material availabi...
2,sales admin,supervisor/koordinator,4 tahun,sarjana ( ),internal sales adminjob description we are ...
3,city operation lead shopee express (cirebon),supervisor/koordinator,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job description responsible for hse implementa...
4,japanese interpreter,pegawai (non-manajemen & non-supervisor),2 tahun,"sertifikat professional, (diploma), (diplo...",overview our clients is manufacture for autom...


In [8]:
df['job_description']=df['job_description'].apply(lambda x: re.sub(' +',' ',x))
df.head()

,job_title,career_level,experience_level,education_level,job_description
0,facility maintenance & smart warehouse manager,manajer/asisten manajer,5 tahun,"sertifikat professional, (diploma), (diplo...",deskripsi pekerjaanrequirements : /si from rep...
1,procurement department head,manajer/asisten manajer,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job role: . responsible for material availabi...
2,sales admin,supervisor/koordinator,4 tahun,sarjana ( ),internal sales & adminjob description :we are ...
3,city operation lead shopee express (cirebon),supervisor/koordinator,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job description:responsible for hse implementa...
4,japanese interpreter,pegawai (non-manajemen & non-supervisor),2 tahun,"sertifikat professional, (diploma), (diplo...",overview: our clients is manufacture for autom...


In [9]:
for index,text in enumerate(df['job_description'][100:104]):
  print('Description %d:\n'%(index+1),text)

Description 1:
 leader teknisi bidang sipildeskripsi pekerjaan:dapat mengelola gedung khusus untuk sipil.dapat menganalisis dan problem solving.memimpin dan mengarahkan sdm yang menjadi pengawasannya baik kinerja dan disiplin kerja.membuat analisa kebutuhan reparasi dan pengadaan baru pada perangkat kerja.kualifikasi:usia maksimal tahun.pendidikan minimal sipil.kesehatan dalam keadaan prima.berpengalaman gedung minimal tahun.dapat memahami fungsi utility gedung (sipil).administrasi, dan mengetahui pekerjaan sipil serta mampu menyusun sop gedung dan melaksanakan preventife maintenance.memiliki kemampuan memimpin, komunikatif, bernegosiasi, bersosialisasi, berfikiri sistematis, dan mampu bekerja dibawah tekanan.
Description 2:
 kualifikasi :min.pengalaman kerja tahun, fresh graduate (welcome)usia maksimal tahunpendidikan ilmu hukumlebih disukai jika kandidat memiliki pengalaman / pemahaman terkait legalitas pertambangan(terutama masalah perizinan).memiliki kepribadian yang baik, integrit

In [11]:
df_new = df.iloc[:100]

translator = Translator(to_lang="en")

def safe_translate(text, chunk_size=500):
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    translated_chunks = []

    for c in chunks:
      try:
        translated_chunks.append(translator.translate(c))
      except:
        # translated_chunks.append(c)
        continue

    return " ".join(translated_chunks)

df_new['deskripsi_pekerjaan_en'] = df_new['job_description'].apply(safe_translate)
df_new.head()

/var/folders/2x/6ysp6yr56c3017xsg0jb_l480000gn/T/ipykernel_8523/77119192.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['deskripsi_pekerjaan_en'] = df_new['job_description'].apply(safe_translate)


,job_title,career_level,experience_level,education_level,job_description,deskripsi_pekerjaan_en
0,facility maintenance & smart warehouse manager,manajer/asisten manajer,5 tahun,"sertifikat professional, (diploma), (diplo...",deskripsi pekerjaanrequirements : /si from rep...,
1,procurement department head,manajer/asisten manajer,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job role: . responsible for material availabi...,
2,sales admin,supervisor/koordinator,4 tahun,sarjana ( ),internal sales & adminjob description :we are ...,
3,city operation lead shopee express (cirebon),supervisor/koordinator,5 tahun,"sarjana ( ), diploma pascasarjana, gelar profe...",job description:responsible for hse implementa...,
4,japanese interpreter,pegawai (non-manajemen & non-supervisor),2 tahun,"sertifikat professional, (diploma), (diplo...",overview: our clients is manufacture for autom...,


In [12]:
df_new.to_csv('clean_data.csv', index=False)

In [13]:
nlp = spacy.load("en_core_web_lg")
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

def extract_doc_node_values(annotation_result):
  if not isinstance(annotation_result, dict) or 'results' not in annotation_result:
    return []

  doc_node_values = []
  results = annotation_result['results']

  if 'full_matches' in results:
    for match in results['full_matches']:
      doc_node_values.append(match['doc_node_value'])

  # if 'ngram_scored' in results:
  #   for match in results['ngram_scored']:
  #     doc_node_values.append(match['doc_node_value'])

  return doc_node_values

df_new['annotations'] = df_new['job_description'].apply(lambda x: skill_extractor.annotate(x))
df_new['skills'] = df_new['annotations'].apply(extract_doc_node_values)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)
/var/folders/2x/6ysp6yr56c3017xsg0jb_l480000gn/T/ipykernel_56094/2391288362.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['annotations'] = df_new['job_description'].apply(lambda x: skill_extractor.annotate(x))
/var/folders/2x/6ysp6yr56c3017xsg0jb_l480000gn/T/ipykernel_56094/2391288362.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [14]:
skill_extractor.describe(df_new.iloc[86]['annotations'])